```cypher
CREATE CONSTRAINT ON (p:Paciente) ASSERT p.id IS UNIQUE;
CREATE CONSTRAINT ON (r:Rastreavel) ASSERT r.id IS UNIQUE;
CREATE CONSTRAINT ON (r:Rastreavel) ASSERT EXISTS(r.tipo);
CREATE CONSTRAINT ON (r:Rastreavel) ASSERT EXISTS(r.nome);
CREATE CONSTRAINT ON ()-[c:CHECKIN]-() ASSERT EXISTS(c.data);

LOAD CSV WITH HEADERS FROM "https://media.githubusercontent.com/media/TiagodePAlves/mc536-projeto/master/dados/flaredown.csv" AS line
WITH line
WHERE line.user_id IS NOT NULL AND line.trackable_id IS NOT NULL
MERGE (p:Paciente {id: line.user_id})
	ON CREATE SET
    	(CASE WHEN line.age IS NOT NULL THEN p END).idade = toInteger(line.age),
    	(CASE WHEN line.sex IS NOT NULL THEN p END).sexo = line.sex,
    	(CASE WHEN line.country IS NOT NULL THEN p END).pais = line.country
MERGE (r:Rastreavel {id: toInteger(line.trackable_id)})
	ON CREATE SET
    	r.tipo = line.trackable_type,
    	r.nome = line.trackable_name
CREATE (p)-[c:CHECKIN {data: datetime(line.checkin_date)}]->(r)
SET (CASE WHEN line.line.trackable_value IS NOT NULL THEN c END).value = line.trackable_value;
```

```cypher
CREATE CONSTRAINT ON (r:Sintoma) ASSERT r.id IS UNIQUE;
CREATE CONSTRAINT ON (r:Sintoma) ASSERT EXISTS(r.tipo);
CREATE CONSTRAINT ON (r:Sintoma) ASSERT EXISTS(r.nome);
CREATE CONSTRAINT ON (r:Condicao) ASSERT r.id IS UNIQUE;
CREATE CONSTRAINT ON (r:Condicao) ASSERT EXISTS(r.tipo);
CREATE CONSTRAINT ON (r:Condica) ASSERT EXISTS(r.nome);
CREATE CONSTRAINT ON (r:Outro) ASSERT r.id IS UNIQUE;
CREATE CONSTRAINT ON (r:Outro) ASSERT EXISTS(r.tipo);
CREATE CONSTRAINT ON (r:Outro) ASSERT EXISTS(r.nome);

MATCH (r:Rastreavel {tipo: 'Symptom'})
SET r:Sintoma;
MATCH (r:Rastreavel {tipo: 'Condition'})
SET r:Condicao;
MATCH (r:Rastreavel)
WHERE NOT r:Sintoma AND NOT r:Condicao
SET r:Outro;
```

```cypher
CREATE CONSTRAINT ON ()-[o:OCORRE_COM]-() ASSERT EXISTS(o.vezes);

MATCH (r1:Rastreavel)<-[:CHECKIN]-(:Paciente)-[:CHECKIN]->(r2:Rastreavel)
WHERE r1.id < r2.id
MERGE (r1)-[o:OCORRE_COM]-(r2)
    ON CREATE SET o.vezes = 1
    ON MATCH SET o.vezes = o.vezes + 1;

CALL algo.unionFind.stream('Rastreavel', 'OCORRE_COM', {})
YIELD nodeId, setId
WITH setId AS grupo, algo.getNodeById(nodeId) AS no
SET no.grupo = grupo
WITH grupo AS maior_grupo, COUNT(no) AS nos
ORDER BY nos DESC
LIMIT 1
MATCH (r:Rastreavel)
WHERE r.grupo <> maior_grupo
DETACH DELETE r;

MATCH (r:Rastreavel)
SET r.grupo = NULL;
```

```cypher
CALL algo.betweenness('Sintoma', 'OCORRE_COM', {direction:'both', write:true, writeProperty:'centralidade'})
YIELD nodes, minCentrality, maxCentrality, sumCentrality, loadMillis, computeMillis, writeMillis;

MATCH (s:Sintoma)
RETURN s.nome, s.centralidade
ORDER BY s.centralidade DESC
LIMIT 10;
```

```cypher
MATCH (c:Condicao)-[:OCORRE_COM]-(r:Rastreavel)
WITH {item: id(c), categories: collect(id(r))} as ligacoesPorCondicao
WITH collect(ligacoesPorCondicao) as dados
CALL algo.similarity.jaccard.stream(dados, {topK: 1, similarityCutoff: 0.0})
YIELD item1, item2, similarity
WHERE item1 < item2
RETURN algo.getNodeById(item1).nome AS c1, algo.getNodeById(item2).nome AS c2, similarity AS similaridade
ORDER BY similaridade DESC
```

```cypher
MATCH (c1:Condicao)-[o1:OCORRE_COM]-(s:Sintoma)-[o2:OCORRE_COM]-(c2:Condicao)
WHERE c1.id < c2.id
MERGE (c1)-[S:SIMILAR]-(c2)
    ON CREATE SET S.peso = toFloat(o1.vezes*o2.vezes)/(1+s.centralidade)
    ON MATCH SET S.peso = S.peso + toFloat(o1.vezes*o2.vezes)/(1+s.centralidade);

MATCH (c1:Condicao)-[s:SIMILAR]-(c2:Condicao)
WHERE c1.id < c2.id
RETURN
	c1.nome, c2.nome, s.peso
ORDER BY s.peso DESC
LIMIT 10;
```

```cypher
CALL algo.louvain('Sintoma', 'OCORRE_COM', {weightProperty:'vezes', write:true, writeProperty:'comunidade'})
YIELD nodes, communityCount, iterations, loadMillis, computeMillis, writeMillis;

MATCH (s:Sintoma)
WITH s.comunidade AS comunidade, COUNT(s) AS nos
ORDER BY nos DESC
LIMIT 1
MATCH (s:Sintoma {comunidade: comunidade})
RETURN s;
```

```cypher
MATCH (t:Outro {tipo: 'Treatment'})-[c:CHECKIN]-(:Paciente)
RETURN t.nome AS tratamento, COUNT(c) AS usos
ORDER BY usos DESC
LIMIT 10;
```